## For 'import ipynb files'
1. install import_ipynb, PyDrive
2. import ipynb files

In [1]:
!pip install -U -q PyDrive
!pip install import_ipynb
import import_ipynb
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# id for import files
'''
layers.ipynb : 1TE_v-Os9bcDepAWODmnDi5WuAz2tyfYn
optim.ipynb : 1kCLfroPeioTdHqogFoOkTVyFYsYe9RAh
'''
# test import
module = drive.CreateFile({'id':'1b1teolDFHMLhucgVFL_-2wjOyFoFzGmJ'})
module.GetContentFile('test_function.ipynb')
from test_function import print_test
print_test()

# import require module
module = drive.CreateFile({'id':'1TE_v-Os9bcDepAWODmnDi5WuAz2tyfYn'})
module.GetContentFile('layers.ipynb')
module = drive.CreateFile({'id':'1kCLfroPeioTdHqogFoOkTVyFYsYe9RAh'})
module.GetContentFile('optim.ipynb')

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=1286e5422a2f7538a484860ed822c0b96fb70b5d744e2dcaa324ba1541830708
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
importing Jupyter notebook from test_function.ipynb
test


### GPU config
- True : cupy
- False : numpy

In [2]:
GPU = True

if GPU:
    import cupy as np
    np.cuda.set_allocator(np.cuda.MemoryPool().malloc)
else:
    import numpy as np

## import requirements

In [3]:
from layers import Convolution, Pooling, FullyConnected
from optim import Adam

# for time series split
!pip install scikit-learn==0.24.2

importing Jupyter notebook from layers.ipynb
importing Jupyter notebook from optim.ipynb
     |████████████████████████████████| 22.3MB 1.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [16]:
# MSE for Loss
def MSE(y, t):
    return 0.5 * np.mean((y-t)**2)

# ReLU activation function
class ReLU:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x<=0)
        y = x.copy()
        y[self.mask] = 0
        return y

    def backward(self, dy):
        dy[self.mask] = 0
        dx = dy
        return dx

# Loss with ReLU
class ReluWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.activation = ReLU()
        self.cache = None

    def forward(self, x, t):
        N, V = x.shape      # batch, output
        
        x = x.reshape(N, V)
        t = t.reshape(N, V)
        x = self.activation.forward(x)

        loss = MSE(x, t)
        self.cache = (t, x, (N, V))
        return loss

    def backward(self, dy=1):
        t, x, (N, V) = self.cache
        dx = dy * (x-t) / N
        
        dx = self.activation.backward(dx)
        dx = dx.reshape(N, V)
        return dx

In [18]:
# CNN Model with Regression
class CnnModelReg:
    def __init__(self, input_dim = (1, 24, 8), 
                 params={'filter_num':30, 'filter_size':5, 
                         'padding':0, 'stride':1},
                 hidden_size=100, output_size=1):
        filter_num = params['filter_num']
        filter_size = params['filter_size']
        padding = params['padding']
        stride = params['stride']

        # not square
        input_size = input_dim[1]
        input_size2 = input_dim[2]
        conv_output_size_h = (input_size - filter_size + 2*padding)/stride + 1
        conv_output_size_w = (input_size2 - filter_size + 2*padding)/stride + 1
        pool_output_size = int(filter_num*(conv_output_size_h/2)*(conv_output_size_w/2))

        self.params = {}
        rand = np.random.randn

        # He initialize
        self.params['cnn_W'] = rand(filter_num, input_dim[0], filter_size, filter_size) / np.sqrt(filter_num/2)
        self.params['cnn_b'] = np.zeros(filter_num)
        self.params['W1'] = rand(pool_output_size, hidden_size)/np.sqrt(pool_output_size/2)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = rand(hidden_size, output_size)/np.sqrt(hidden_size/2)
        self.params['b2'] = np.zeros(output_size)

        self.layers=[
                     Convolution(self.params['cnn_W'], self.params['cnn_b'], stride, padding),
                     ReLU(),
                     Pooling(pool_h=2, pool_w=2, stride=2),
                     FullyConnected(self.params['W1'], self.params['b1']),
                     ReLU(),
                     FullyConnected(self.params['W2'], self.params['b2']),
        ]
        self.loss_layer = ReluWithLoss()

        self.grads = []
        self.params = []
        for i in [0, 3, 5]:
            self.params += self.layers[i].params
            self.grads+=self.layers[i].grads

    def predict(self, x):
        x = np.array(x)
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def forward(self, x, t):
        x = np.array(x)
        t = np.array(t)
        x = self.predict(x)
        loss = self.loss_layer.forward(x, t)
        return loss

    def backward(self, dy=1):
        dy = self.loss_layer.backward(dy)

        for layer in reversed(self.layers):
            dy = layer.backward(dy)
        return dy
    
    def fit(self, train_X=None, train_y=None,learning_rate=0.01, epochs=10, batch_size=32, verbose=0):
        optimizer = Adam(learning_rate)

        data_size = train_X.shape[0]
        max_iters = data_size//batch_size

        for epoch in range(1, epochs+1):
            # shuffle train data
            idx = numpy.random.permutation(numpy.arange(data_size))
            x_data = train_X[idx]
            y_data = train_y[idx]

            epoch_loss = 0
            start_time = time.time()
            for iter in range(max_iters):
                batch_x = x_data[iter*batch_size:(iter+1)*batch_size]
                batch_y = y_data[iter*batch_size:(iter+1)*batch_size]

                loss = self.forward(batch_x, batch_y)
                self.backward()
                params, grads = self.params, self.grads
                optimizer.update(params, grads)

                epoch_loss += loss
            avg_loss = epoch_loss/max_iters

            if verbose:
                duration = start_time-time.time()
                print(f'epoch:{epoch}/{epochs}, 시간:{duration:.2f}[s], loss:{avg_loss:.5f}')

In [28]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

# configuration setting
def model_config():
    # parameter for CNN Model
    filter_num = [30]
    filter_size = [3, 5]
    epochs = [10]
    batch_size = [64]
    learning_rate = [0.1, 0.01]
    
    # create config data
    configs = []
    for i in filter_num:
        for j in filter_size:
            for k in epochs:
                for l in batch_size:
                    for m in learning_rate:
                        config = [i, j, k, l, m]
                        configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_filter, n_fsize, n_epochs, n_batch, learning_rate = config
    cnn_params = {'filter_num':n_filter, 'filter_size':n_fsize, 
                  'stride':1, 'padding':0}
    model = CnnModelReg(params=cnn_params)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    t = np.array(t)
    return np.mean((x-t)**2)

In [30]:
# dataset
import pandas as pd
import numpy
import time
from datetime import datetime

df_parser = lambda x: datetime.strptime(x, '%Y %m %d %H')    # string to datetime
# data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv'
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv'
df = pd.read_csv(data_url, sep=',', parse_dates=[['year', 'month', 'day', 'hour']], date_parser=df_parser, index_col=0)

del df['No']
df.columns = ['pm2.5', 'dewp', 'temp', 'pres', 'cbwd','wind_speed', 'snow', 'rain']
df = df[24:]            # NaN values in first 24hours

# sklearn library for time series split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error

dataset = df.values
label_encoder = LabelEncoder()
dataset[:, 4] = label_encoder.fit_transform(dataset[:, 4])  # for wind direction
dataset = dataset.astype('float')

n_inputs = 24       # input time dim
n_features = 8      # input feature dim
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)       # time series to img data
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)

n_splits = 3
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []
best_error = []
i = 1

print('config : filter_num, filter_size, epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, 1, n_inputs, n_features)
    val_X = val_X.reshape(-1, 1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1, 1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, 1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        predicted = model.predict(val_X)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f'error(rmse):{error.item():.2f}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    # outer
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    predicted = selected_model.predict(test_cv_X)
    error = np.sqrt(MSE_metric(predicted, test_cv_y))
    best_error.append(error)

    # model eval
    print(f'best_model => error(rmse) : {error.item():.2f}, param:{configs[param]}')

config : filter_num, filter_size, epochs, batch_size, learning_rate
fold : 1/3
 == train [30, 3, 10, 64, 0.1] model ==  error(rmse):37.99
 == train [30, 3, 10, 64, 0.01] model ==  error(rmse):35.66
 == train [30, 5, 10, 64, 0.1] model ==  error(rmse):89.52
 == train [30, 5, 10, 64, 0.01] model ==  error(rmse):142.59
best_model => error(rmse) : 26.41, param:[30, 3, 10, 64, 0.01]
fold : 2/3
 == train [30, 3, 10, 64, 0.1] model ==  error(rmse):128.21
 == train [30, 3, 10, 64, 0.01] model ==  error(rmse):26.70
 == train [30, 5, 10, 64, 0.1] model ==  error(rmse):85.51
 == train [30, 5, 10, 64, 0.01] model ==  error(rmse):27.15
best_model => error(rmse) : 28.64, param:[30, 3, 10, 64, 0.01]
fold : 3/3
 == train [30, 3, 10, 64, 0.1] model ==  error(rmse):40.34
 == train [30, 3, 10, 64, 0.01] model ==  error(rmse):30.16
 == train [30, 5, 10, 64, 0.1] model ==  error(rmse):82.20
 == train [30, 5, 10, 64, 0.01] model ==  error(rmse):129.89
best_model => error(rmse) : 22.61, param:[30, 3, 10, 64,

In [35]:
model_evaluation = sum(best_error)
model_evaluation /= n_splits
print(f'evaluation [Mean RMSE] : {model_evaluation}')

evaluation [Mean RMSE] : 25.888455177055764


In [38]:
predicted = selected_model.predict(test_cv_X)
print(f'MSE : {MSE_metric(predicted, test_cv_y)}')
print(f'RMSE : {np.sqrt(MSE_metric(predicted, test_cv_y))}')

def MAE_metric(x, t):
    t = np.array(t)
    return np.mean(numpy.abs(x-t))
print(f'MAE : {MAE_metric(predicted, test_cv_y)}')

MSE : 511.4273951674669
RMSE : 22.614760559587335
MAE : 13.7427807869428
